In [23]:
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification

from google.colab import drive
drive.mount('/content/drive')

# Şikayet modelini yükle
sikayet_model_path = "/content/drive/MyDrive/yorumlar/sikayet_eğitim_modeli"
sikayet_model = BertForSequenceClassification.from_pretrained(sikayet_model_path)
sikayet_tokenizer = BertTokenizer.from_pretrained(sikayet_model_path)
sikayet_model.eval()

# Kategori modelini yükle
kategori_model_path = "/content/drive/MyDrive/yorumlar/bert_kategori_egitim_modeli"
kategori_model = BertForSequenceClassification.from_pretrained(kategori_model_path)
kategori_tokenizer = BertTokenizer.from_pretrained(kategori_model_path)
kategori_model.eval()

# Etiketleri yükle
etiketler = np.load("/content/drive/MyDrive/yorumlar/bert_kategori_egitim_modeli/kategori_etiketleri.npy", allow_pickle=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
yorumlar = [
    "Otobüsler çok düzensiz çalışıyor.",
    "ASKİ'ye çok teşekkür ederim, çok ilgilendiler.",
    "Yollar çok kötü durumda, araçlar zarar görüyor.",
    "Lambalar yanmıyor, geceleri çok karanlık.",
    "Müzik sesi çok yüksek, rahatsız edici.",
    "Çöpler haftalardır alınmıyor."
]


In [25]:
for yorum in yorumlar:
    # Şikayet tahmini
    inputs = sikayet_tokenizer(yorum, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        output = sikayet_model(**inputs)
        tahmin = torch.argmax(output.logits, dim=1).item()

    if tahmin == 1:
        sikayet_text = "Şikayet"
        # Şikayet olanlara kategori tahmini
        kat_inputs = kategori_tokenizer(yorum, return_tensors="pt", truncation=True, padding=True, max_length=128)
        with torch.no_grad():
            kat_output = kategori_model(**kat_inputs)
            kat_id = torch.argmax(kat_output.logits, dim=1).item()
            kategori = etiketler[kat_id]
    else:
        sikayet_text = "Şikayet Değil"
        kategori = "Teşekkür"

    # Sonucu yazdır
    print(f"Yorum: {yorum}")
    print(f"Tahmin: {sikayet_text} | Kategori: {kategori}")
    print("-" * 60)


Yorum: Otobüsler çok düzensiz çalışıyor.
Tahmin: Şikayet | Kategori: Diğer
------------------------------------------------------------
Yorum: ASKİ'ye çok teşekkür ederim, çok ilgilendiler.
Tahmin: Şikayet Değil | Kategori: Teşekkür
------------------------------------------------------------
Yorum: Yollar çok kötü durumda, araçlar zarar görüyor.
Tahmin: Şikayet | Kategori: Ulaşım
------------------------------------------------------------
Yorum: Lambalar yanmıyor, geceleri çok karanlık.
Tahmin: Şikayet | Kategori: Aydınlatma
------------------------------------------------------------
Yorum: Müzik sesi çok yüksek, rahatsız edici.
Tahmin: Şikayet | Kategori: Diğer
------------------------------------------------------------
Yorum: Çöpler haftalardır alınmıyor.
Tahmin: Şikayet | Kategori: Diğer
------------------------------------------------------------
